# Production-Ready Inference
The production-ready inference Jupyter Notebook is meant to allow a user-friendly inference that can be run in local, without the need to struggle for understanding deeply what is going on under the hood of our model and trying to find a way out from our hierarchy!

Before running this notebook, make sure that you have all of these things in place:
1. You have inserted the images for which you want your annotations to be predicted inside the **INFERENCE_DIR** folder.
2. You have added a file named **annotations.csv** to the **ANNOTATIONS&PREDICTIONS** folder, if you want your ground-truth annotations to be compared to the predictions made by our model.

# SEE WHETHER TO ADD HERE THE CHECKPOINTS ALREADY!

In [ ]:
import os